In [1]:
import numpy as np
import matplotlib.pyplot as plt

El error del articulo radica en que al realizar la parte del predictor la indexación de la aceleración debería ser
en el tiempo actual, pero en las ecuaciones la indexación de a parece incorrecta:

$ r_{n+1} = r_n + hv_n + h^2 \sum_{p=1}^{q-1} b_p a_{n-q+1} $

$ r_{n+1} = r_n + hv_n + h^2 \sum_{p=1}^{q-1} c_p a_{n-q+2} $

$ hv_{n+1} = r_{n+1} - r_n + h^2 \sum_{p=1}^{q-1} c_p a_{n-q+2} $
Mientras las ecuaciones deberían ser así: 

$ r_{n+1} = r_n + hv_n + h^2 \sum_{p=1}^{q-1} b_p a_{n-p+1} $

$ r_{n+1} = r_n + hv_n + h^2 \sum_{p=1}^{q-1} c_p a_{n-p+2} $

$ hv_{n+1} = r_{n+1} - r_n + h^2 \sum_{p=1}^{q-1} c_p a_{n-p+2} $

In [2]:
def calculate_acceleration(r, m, e_over_sigma, sigma):
    """
    Calcula a con potencial de Lennard-Jones.
    """
    N = len(r) // 3  # Número de moléculas
    a = [0] * len(r)
    
    for i in range(N):
        a_i = [0, 0, 0]
        for j in range(N):
            if i != j:
                r_ij = [r[i*3+k] - r[j*3+k] for k in range(3)]
                r_ij_norm = sum([x ** 2 for x in r_ij]) ** 0.5
                force_magnitude = 48 * e_over_sigma * ((sigma / r_ij_norm) ** 13 - 0.5 * (sigma / r_ij_norm) ** 7)
                for k in range(3):
                    a_i[k] += force_magnitude * r_ij[k] / r_ij_norm
        for k in range(3):
            a[i*3+k] = a_i[k] / m
    return a

def calculate_position(r_prev, v, a_prev, a_current, h):
    r_new = []
    for i in range(len(r_prev) // 3):
        index = i * 3
        r_new.append(r_prev[index] + h * v[index] + (h ** 2 / 6) * (4 * a_current[index] - a_prev[index]))
        r_new.append(r_prev[index + 1] + h * v[index + 1] + (h ** 2 / 6) * (4 * a_current[index + 1] - a_prev[index + 1]))
        r_new.append(r_prev[index + 2] + h * v[index + 2] + (h ** 2 / 6) * (4 * a_current[index + 2] - a_prev[index + 2]))
    return r_new


def calculate_velocity(v_prev, a_prev, a_current, a_next, h):
    v_new = []
    for v_i, a_prev_i, a_current_i, a_next_i in zip(v_prev, a_prev, a_current, a_next):
        v_new.append(v_i + (h / 6) * (2 * a_next_i + 5 * a_current_i - a_prev_i))
    return v_new

In [3]:
def beeman_predictor_corrector(r, v, a, h, m, e_over_sigma, sigma):
    """
    Implementa el algoritmo predictor-corrector de Beeman para integración numérica.
    """
    N = len(r) // 3  # Número de moléculas
    
    #predicción
    r_predicted = [0] * len(r)
    v_predicted = [0] * len(v)
    
    
    for i in range(N):
        r_prev = r[i*3:i*3+3]
        v_prev = v[i*3:i*3+3]
        a_prev = a[i*3:i*3+3]
        a_current = a[i*3+3:i*3+6] if i < N - 1 else [0, 0, 0]  # Último paso, no hay aceleración futura
        r_predicted[i*3:i*3+3] = calculate_position(r_prev, v_prev, a_prev, a_current, h)
        v_predicted[i*3:i*3+3] = calculate_velocity(v_prev, a_prev, a_current, a[i*3+6:i*3+9], h)
    
    #corrección
    for i in range(N):
        a_predicted = [0, 0, 0] if i == N - 1 else a[(i+1)*3:(i+1)*3+3]
        a_corrected = a_predicted
        if i > 0:
            a_corrected = [(a_predicted[j] + 2 * a[i*3+j] - a_prev[j]) / 3 for j in range(3)]
        a_prev = a[i*3:i*3+3]
        v[i*3:i*3+3] = calculate_velocity(v[i*3:i*3+3], a_prev, a_predicted, a_corrected, h)
        r[i*3:i*3+3] = calculate_position(r[i*3:i*3+3], v[i*3:i*3+3], a_prev, a_corrected, h)
    return r, v

In [ ]:
N = 1000  # N
h = 0.01  # Paso de tiempo
m = 1.0   # Masa
e_over_sigma = 1.0  # Energía de Lennard-Jones dividida por sigma
sigma = 1.0  #sigma del potencial de Lennard-Jones
r = np.random.rand(N * 3)  # Posiciones aleatorias entre 0 y 1
v = np.zeros(N * 3)  # Velocidades iniciales
a = calculate_acceleration(r, m, e_over_sigma, sigma)  # Calcular aceleraciones iniciales
timesteps = 1000
positions = []
velocities = []
times = []
for _ in range(timesteps):
    r, v = beeman_predictor_corrector(r, v, a, h, m, e_over_sigma, sigma)
    positions.append(r.copy())
    velocities.append(v.copy())
    times.append(_ * h)

positions = np.array(positions)
velocities = np.array(velocities)
times = np.array(times)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(N):
    plt.plot(times, positions[:, i*3])
plt.xlabel('Tiempo')
plt.ylabel('Posicion')
plt.title('Posicion vs Tiempo para cada molecula')
plt.legend()
plt.grid(True)
plt.show()
plt.show()